In [3]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

class CNN(nn.Module): # From JYKim
    def __init__(self):
        super().__init__()
        
        # Original code from JYKim
        self.layer = nn.Sequential(
            # Input channel: Number of input state map: Pressure / Water saturation
            # 2 * 15 * 15
            nn.Conv2d(in_channels=2, out_channels=32, kernel_size=(3, 3), padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # 32 * 15 * 15
            nn.AvgPool2d(stride=2, kernel_size=(2, 2)),

            # 32 * 7 * 7
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # 64 * 7 * 7
            nn.AvgPool2d(stride=2, kernel_size=(2, 2)),

            # 64 * 3 * 3
        )

        self.layer.apply(self._init_weight)

        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 3 * 3, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Dropout(0.4),

            nn.Linear(128, 1),
        )

    def forward(self, x):
        out = self.layer(x)
        out = torch.nn.Flatten()(out)
        out = self.fc_layer(out)
        return out

    def _init_weight(self, layer):
        if isinstance(layer, nn.Conv2d):
            torch.nn.init.xavier_uniform(layer.weight)

cuda
